# Official Channel Video Crawling

In [ ]:
import requests
import os
from dotenv import load_dotenv
import time
import re
import pandas as pd
load_dotenv()

True

In [5]:
channels = [
    {
        "label": "ISEGYE_IDOL",
        "handle": "isegyeidol_ofcl"
    },
    {
        "label": "PLAVE",
        "handle": "plave_official"
    },
    {
        "label": "IVE",
        "handle": "IVEstarship"
    },
    {
        "label": "RIIZE",
        "handle": "RIIZE_official"
    }
]

In [3]:
# 환경변수에서 YouTube API Key 불러오기
API_KEY = os.getenv("YOUTUBE_API_KEY")

# 채널 handle(@ 뒤 문자열)를 이용해 channelId를 가져오는 함수
def get_channel_id_from_handle(handle):
    
    # YouTube channel handle을 기반으로 channelId를 조회한다.

    # Parameters
    # ----------
    # handle : str
    # 유튜브 채널 핸들명 (@ 뒤 문자열)

    # Returns
    # -------
    # channel_id : str
    # YouTube 내부에서 사용하는 채널 고유 ID
    
    url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",       # 기본 정보 요청
        "q": handle,             # 검색어로 handle 사용
        "type": "channel",       # 채널만 검색
        "maxResults": 1,         # 가장 상단 결과 1개만 사용
        "key": API_KEY
    }

    # API 요청
    resp = requests.get(url, params=params)
    data = resp.json()

    items = data.get("items", [])
    if not items:
        raise ValueError(f"Channel not found for handle: {handle}")
    
    # 검색 결과에서 channelId 추출
    channel_id = items[0]["snippet"]["channelId"]
    return channel_id

In [18]:
# 각 채널 handle에 대해 channelId가 잘 나오는지 확인
for c in channels:
    cid = get_channel_id_from_handle(c["handle"])
    print(c["label"], cid)

ISEGYE_IDOL UCmblbE_WxTaFWcMEGFJ_97w
PLAVE UCPZIPuQPrfrUG9Xe_okEmQA
IVE UC-Fnix71vRP64WXeo0ikd0Q
RIIZE UCdVD0MsYecQaIE5Ru-pOIQQ


In [17]:
def get_videos_from_uploads_playlist(
    uploads_playlist_id: str,
    channel_label: str,
    channel_handle: str,
    sleep_sec: float = 0.2
):
    
    # uploads playlistId를 이용해 채널에 업로드된 모든 영상의
    # video_id, 제목, 게시일을 수집한다.

    # Parameters
    # ----------
    # uploads_playlist_id : str
    #     채널의 uploads playlist ID
    # channel_label : str
    #     분석용 채널 라벨 (예: ISEGYE_IDOL)
    # channel_handle : str
    #     채널 handle (@ 뒤 문자열)
    # sleep_sec : float
    #     API 요청 간 대기 시간 (quota 안정성)

    # Returns
    # -------
    # videos : list[dict]
    #     채널의 모든 영상 기본 메타데이터 리스트
    

    videos = []            # 수집된 영상 정보를 저장할 리스트
    page_token = None      # pagination을 위한 토큰
    page_count = 0         # 페이지 수 (로그용)

    while True:
        url = "https://www.googleapis.com/youtube/v3/playlistItems"
        params = {
            "part": "snippet",
            "playlistId": uploads_playlist_id,
            "maxResults": 50,      # playlistItems API 최대
            "key": API_KEY
        }

        # 다음 페이지가 있을 경우 pageToken 추가
        if page_token:
            params["pageToken"] = page_token

        # API 요청
        resp = requests.get(url, params=params, timeout=10)
        data = resp.json()

        items = data.get("items", [])
        if not items:
            break

        # 각 영상의 기본 정보 추출
        for item in items:
            snippet = item.get("snippet", {})
            resource = snippet.get("resourceId", {})

            video_id = resource.get("videoId")
            if not video_id:
                continue

            videos.append({
                "channel_label": channel_label,
                "channel_handle": channel_handle,
                "video_id": video_id,
                "video_title": snippet.get("title"),
                "published_at": snippet.get("publishedAt")
            })

        page_count += 1
        print(f"[{channel_label}] page {page_count} | total videos: {len(videos)}")

        # 다음 페이지 토큰
        page_token = data.get("nextPageToken")
        if not page_token:
            break

        time.sleep(sleep_sec)

    return videos

In [16]:
def get_uploads_playlist_id(channel_id: str) -> str:
    
    # YouTube channelId를 기반으로 해당 채널의 uploads playlistId를 조회한다.
    # uploads playlist에는 채널에 업로드된 모든 영상이 포함되어 있다.

    # Parameters
    # ----------
    # channel_id : str
    #     YouTube 채널 고유 ID (UC로 시작)

    # Returns
    # -------
    # uploads_playlist_id : str
    #     채널의 모든 영상이 담긴 uploads playlist ID
    

    url = "https://www.googleapis.com/youtube/v3/channels"
    params = {
        "part": "contentDetails",  # 플레이리스트 정보 포함
        "id": channel_id,
        "key": API_KEY
    }

    # API 요청 (timeout 필수)
    resp = requests.get(url, params=params, timeout=10)
    data = resp.json()

    items = data.get("items", [])
    if not items:
        raise ValueError(f"No channel data found for channel_id={channel_id}")

    # uploads playlist ID 추출
    uploads_playlist_id = (
        items[0]
        ["contentDetails"]
        ["relatedPlaylists"]
        ["uploads"]
    )

    return uploads_playlist_id

In [19]:
all_channel_videos = []

for channel in channels:
    label = channel["label"]
    handle = channel["handle"]

    print(f"\n=== Fetching videos for {label} ===")

    # 1. handle → channel_id
    channel_id = get_channel_id_from_handle(handle)

    # 2. channel_id → uploads playlist id
    uploads_pid = get_uploads_playlist_id(channel_id)

    # 3. uploads playlist → 영상 목록 수집
    videos = get_videos_from_uploads_playlist(
        uploads_playlist_id=uploads_pid,
        channel_label=label,
        channel_handle=handle
    )

    print(f"{label} videos collected: {len(videos)}")

    # 4. 전체 리스트에 병합
    all_channel_videos.extend(videos)

# 전체 영상 개수 확인
len(all_channel_videos)


=== Fetching videos for ISEGYE_IDOL ===
[ISEGYE_IDOL] page 1 | total videos: 50
[ISEGYE_IDOL] page 2 | total videos: 89
ISEGYE_IDOL videos collected: 89

=== Fetching videos for PLAVE ===
[PLAVE] page 1 | total videos: 50
[PLAVE] page 2 | total videos: 100
[PLAVE] page 3 | total videos: 150
[PLAVE] page 4 | total videos: 200
[PLAVE] page 5 | total videos: 250
[PLAVE] page 6 | total videos: 300
[PLAVE] page 7 | total videos: 350
[PLAVE] page 8 | total videos: 400
[PLAVE] page 9 | total videos: 450
[PLAVE] page 10 | total videos: 500
[PLAVE] page 11 | total videos: 550
[PLAVE] page 12 | total videos: 600
[PLAVE] page 13 | total videos: 650
[PLAVE] page 14 | total videos: 700
[PLAVE] page 15 | total videos: 750
[PLAVE] page 16 | total videos: 800
[PLAVE] page 17 | total videos: 850
[PLAVE] page 18 | total videos: 900
[PLAVE] page 19 | total videos: 950
[PLAVE] page 20 | total videos: 1000
[PLAVE] page 21 | total videos: 1050
[PLAVE] page 22 | total videos: 1100
[PLAVE] page 23 | total vi

5157

In [27]:
def parse_duration_to_seconds(duration):
    """
    ISO 8601 형식의 YouTube duration을 초 단위로 변환
    예:
    - PT45S → 45
    - PT1M12S → 72
    - PT1H3M → 3780
    """
    if not duration or not isinstance(duration, str):
        return None
    
    pattern = re.compile(
        r'PT'
        r'(?:(\d+)H)?'
        r'(?:(\d+)M)?'
        r'(?:(\d+)S)?'
    )

    match = pattern.match(duration)
    if not match:
        return None

    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0

    return hours * 3600 + minutes * 60 + seconds

In [32]:
def fetch_video_statistics_batch(video_ids):
    """
    여러 video_id를 batch로 받아
    조회수 / 좋아요 / 댓글 수 / 영상 길이 / 숏폼 여부를 반환

    Parameters
    ----------
    video_ids : list[str]
        video_id 리스트 (최대 50개)

    Returns
    -------
    dict
        {
          video_id: {
              view_count,
              like_count,
              comment_count,
              duration_sec,
              video_type
          }
        }
    """
    url = "https://www.googleapis.com/youtube/v3/videos"

    params = {
        "part": "statistics,contentDetails",
        "id": ",".join(video_ids),  # ⭐ batch 처리 핵심
        "key": os.getenv("YOUTUBE_API_KEY")
    }

    response = requests.get(url, params=params)
    data = response.json()

    stats_map = {}

    for item in data.get("items", []):
        vid = item["id"]

        statistics = item.get("statistics", {})
        content = item.get("contentDetails", {})

        # 조회수 / 좋아요 / 댓글 수
        view_count = int(statistics.get("viewCount", 0))
        like_count = int(statistics.get("likeCount", 0))
        comment_count = int(statistics.get("commentCount", 0))

        # 영상 길이 (숏폼 판단용)
        duration = content.get("duration")
        duration_sec = parse_duration_to_seconds(duration)

        # Shorts 판별
        if duration_sec is None:
            video_type = "UNKNOWN"
        elif duration_sec < 60:
            video_type = "SHORT"
        else:
            video_type = "LONG"

        stats_map[vid] = {
            "view_count": view_count,
            "like_count": like_count,
            "comment_count": comment_count,
            "duration_sec": duration_sec,
            "video_type": video_type
        }

    return stats_map

In [29]:
video_ids = [v["video_id"] for v in all_channel_videos]

In [30]:
def chunked(lst, size=50):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

In [33]:
video_stats = {}

for chunk in chunked(video_ids, 50):
    batch_stats = fetch_video_statistics_batch(chunk)
    video_stats.update(batch_stats)

In [34]:
for v in all_channel_videos:
    stats = video_stats.get(v["video_id"], {})

    v["view_count"] = stats.get("view_count")
    v["like_count"] = stats.get("like_count")
    v["comment_count"] = stats.get("comment_count")
    v["duration_sec"] = stats.get("duration_sec")
    v["video_type"] = stats.get("video_type")

In [35]:
# all_channel_videos → DataFrame 변환
df_videos = pd.DataFrame(all_channel_videos)

# 안전장치: 조회수 없는 영상 제거
df_videos = df_videos.dropna(subset=["view_count"])

# 채널별 조회수 Top-N 영상 추출 함수
def select_top_videos(df, n=3):
    return (
        df
        .sort_values("view_count", ascending=False)
        .groupby("channel_label")
        .head(n)
        .reset_index(drop=True)
    )

top_videos = select_top_videos(df_videos, n=3)
top_videos[["channel_label", "video_title", "view_count", "video_type"]]

NameError: name 'pd' is not defined

In [ ]:
all_top_comments = []

for i, row in top_videos.iterrows():
    video_id = row["video_id"]
    title = row["video_title"]
    channel = row["channel_label"]

    print(f"[{channel}] Fetching comments: {title}")

    comments = fetch_all_comments(video_id)

    # 댓글에 영상/채널 메타 붙이기
    for c in comments:
        c["channel_label"] = channel
        c["video_title"] = title
        c["video_type"] = row["video_type"]

    all_top_comments.extend(comments)

len(all_top_comments)

In [ ]:
df_videos.groupby(["channel_label", "video_type"]).size()

In [ ]:
df_videos.groupby(
    ["channel_label", "video_type"]
).agg(
    avg_view=("view_count", "mean"),
    median_view=("view_count", "median"),
    video_count=("video_id", "count")
).reset_index()


In [ ]:
(
    df_videos
    .groupby("channel_label")["video_type"]
    .value_counts(normalize=True)
    .rename("ratio")
    .reset_index()
)

In [ ]:
df_comments = pd.DataFrame(all_top_comments)
df_comments.to_csv("youtube_top_comments.csv", index=False)